In [ ]:
# pip install pyarabic
# pip install googletrans==4.0.0-rc1
# pip install openpyxl
# pip install -U deep-translator

In [1]:
import pandas as pd
import pyarabic.araby as araby
import numpy as np
from googletrans import Translator
from deep_translator import GoogleTranslator
import time
import tqdm
import json
import random

In [2]:
pd.set_option("display.max_columns", None)

# TRANSLATE DATA FROM AR TO EN

In [ ]:
# READ ORIGINAL DATA
data = pd.read_excel(
    r"C:\Users\sanav\work\D2R\NLO\University\public\data\UNI-IND-2022-tamher.xlsx",
    dtype="object",
)
df.dropna(inplace=True)

df = df[
    [
        "IndicatorCode",
        "IndicatorDescription",
        "Nationality",
        "Gender",
        "Graduation Year",
        "EducationLevel",
        "GeneralMajorName",
        "NarrowMajorName",
        "MajorCodeByClassification",
        "MajorNameByClassification",
        "GOSIoccupationDescription",
        "ISCOOccupationDescription",
        "IsMatched",
        "Employment Information Source",
        "PeriodToEmployment",
        "IndicatorValue"
    ]
]

In [ ]:
# translation code
def translate_text(df, column, translation_results):
    a_to_e_translated = {}
    unique_values = df[column].unique().tolist()

    for value in tqdm.tqdm(unique_values, total=len(unique_values), desc=f"Translating {column}..."):
        if isinstance(value, str):
            if value not in a_to_e_translated:
                success = False
                retries = 3
                while not success and retries > 0:
                    try:
                        translator = GoogleTranslator(source='auto', target='en')
                        translated_text = translator.translate(value)
                        a_to_e_translated[value] = translated_text
                        success = True
                    except Exception as e:
                        print(f"Error translating '{value}': {e}. Retrying in 1 minute...")
                        time.sleep(60)
                        retries -= 1
                if not success:
                    print(f"Failed to translate '{value}' after retries.")
                    a_to_e_translated[value] = value
        else:
            a_to_e_translated[value] = str(value)
    translation_results[column] = a_to_e_translated
    df[f'{column}'] = df[column].map(lambda x: a_to_e_translated.get(x, x))

columns_to_translate = [
        "ISCOOccupationDescription"
        
]

translation_results = {}
for column in columns_to_translate:
    translate_text(df, column, translation_results)

In [ ]:
trs = GoogleTranslator(source='auto', target='en')
trs.translate("برامج غير معروفة")

In [ ]:
output_excel_path = r"C:\Users\sanav\work\D2R\NLO\University\public\output\ar_to_en_translations.xlsx"
with pd.ExcelWriter(output_excel_path, engine='xlsxwriter') as writer:
    for sheet_name, key_value_pairs in translation_results.items():
        df = pd.DataFrame(list(key_value_pairs.items()), columns=['Key', 'Value'])
        df.to_excel(writer, index=False, sheet_name=sheet_name)

# DATA TRANSLATION

In [3]:
# READ ORIGINAL DATA
data = pd.read_excel(
    r"C:\Users\sanav\work\D2R\NLO\University\public\data\UNI-IND-2022-tamher.xlsx",
    dtype="object",
)

In [4]:
df = data.copy()

In [5]:
df.shape

(253883, 16)

In [6]:
df.dropna(inplace=True)

df = df[
    [
        "IndicatorDescription",
        "IndicatorValue",
        "Nationality",
        "Gender",
        "Graduation Year",
        "EducationLevel",
        "GeneralMajorName",
        "NarrowMajorName",
        "MajorNameByClassification",
        # "GOSIoccupationDescription",
        "ISCOOccupationDescription",
        "PeriodToEmployment",
    ]
]

In [7]:
df.columns

Index(['IndicatorDescription', 'IndicatorValue', 'Nationality', 'Gender',
       'Graduation Year', 'EducationLevel', 'GeneralMajorName',
       'NarrowMajorName', 'MajorNameByClassification',
       'ISCOOccupationDescription', 'PeriodToEmployment'],
      dtype='object')

In [8]:
# Code for changing values from Arabic to English in original data
translated_results = {}
excel_data = pd.read_excel(r"C:\Users\sanav\work\D2R\NLO\University\public\data\reflected.xlsx", sheet_name=None)

for sheet_name, sheet_data in excel_data.items():
    translated_results[sheet_name] = {
        str(key).strip(): str(value).strip() if isinstance(value, str) else value
        for key, value in zip(sheet_data.iloc[:, 0], sheet_data.iloc[:, 1])
    }

for column, translation_dict in translated_results.items():
    if column in df.columns:
        if df[column].dtype == 'object':
            df[column] = df[column].str.strip()
        df[column] = df[column].map(translation_dict).fillna(df[column])

In [11]:
df = df[df['Nationality'] == 'Saudi']
df.reset_index(drop=True, inplace=True)
df.replace(['N/A', 'N.A.', np.nan], None, inplace=True)

In [12]:
df.columns

Index(['IndicatorDescription', 'IndicatorValue', 'Nationality', 'Gender',
       'Graduation Year', 'EducationLevel', 'GeneralMajorName',
       'NarrowMajorName', 'MajorNameByClassification',
       'ISCOOccupationDescription', 'PeriodToEmployment'],
      dtype='object')

In [14]:
# ignore_values = {
#     "Gender": ['Unclassified'],
#     'EducationLevel' : ['Unclassified'],
#     'GeneralMajorName': ['Unknown programs'],
#     'NarrowMajorName': ['Unclassified programs'],
#     "Major": [
#         'Unknown Specializations',
#         'Unspecified Specialization in Social and Behavioral Sciences',
#         'Unspecified Specialization in Humanities (Excluding Languages)',
#         'Unspecified Specialization in Business, Administration, and Law',
#         'Unspecified Specialization in Health',
#         'Unspecified Specialization in Physical Sciences',
#         'Unspecified Specialization in Security Services',
#         'Unspecified Specialization in Business and Management'
#     ],
#     'ISCOOccupationDescription': [0]
# }

# ignore_values = {
#     "Gender": ['Unclassified'],
#     'EducationLevel' : ['Unclassified'],
#     'GeneralMajorName': ['Unknown programs'],
#     'NarrowMajorName': ['Unclassified programs'],
#     "Major": [
#         'Unknown Specializations'
#     ],
#     'ISCOOccupationDescription': [0]
# }

In [13]:
# # Remove rows containing the specified values
# for column, values in ignore_values.items():
#     if column in df.columns:
#         df = df[~df[column].isin(values)]


# # Replace specified values with None
# for column, values in ignore_values.items():
#     if column in df.columns:
#         df[column] = df[column].replace(values, None)

df.reset_index(drop=True, inplace=True)

In [14]:
df.shape

(253883, 11)

In [16]:
df.to_json(r"C:\Users\sanav\work\D2R\NLO\University\public\output\translated_full_data.json", orient="records", indent=4, force_ascii=False)
df.to_excel(r"C:\Users\sanav\work\D2R\NLO\University\public\output\translated_full_data.xlsx", index=False)

# TRANSLATE FROM EN TO AR

In [33]:
df = pd.read_excel(r"C:\Users\sanav\work\D2R\NLO\University\public\output\translated_full_data.xlsx", dtype='object')

In [34]:
df.shape

(253883, 11)

In [38]:
unique_values = df['MajorNameByClassification'].unique().tolist()

# Specify the file name
file_name = r"C:\Users\sanav\work\D2R\NLO\University\public\output\MajorNameByClassification.txt"

# Write the list to the file
with open(file_name, 'w', encoding='utf-8') as file:
    for item in unique_values:
        file.write(f"{item}\n")

print(f"Unique values have been written to {file_name}")


Unique values have been written to C:\Users\sanav\work\D2R\NLO\University\public\output\MajorNameByClassification.txt


In [19]:
df['Gender'].value_counts()

Gender
Male            127018
Female          126863
Unclassified         2
Name: count, dtype: int64

In [20]:
# Code for changing values from Arabic to English in original data
translated_results = {}
excel_data = pd.read_excel(r"C:\Users\sanav\work\D2R\NLO\University\public\data\reflected.xlsx", sheet_name=None)

for sheet_name, sheet_data in excel_data.items():
    # Swap the key-value pairs - now English will be key and Arabic will be value
    translated_results[sheet_name] = {
        str(value).strip(): str(key).strip() if isinstance(key, str) else key
        for key, value in zip(sheet_data.iloc[:, 0], sheet_data.iloc[:, 1])
    }

# Apply the translations
for column, translation_dict in translated_results.items():
    if column in df.columns:
        if df[column].dtype == 'object':
            df[column] = df[column].str.strip()
        df[column] = df[column].map(translation_dict).fillna(df[column])


In [25]:
df.columns

Index(['IndicatorDescription', 'IndicatorValue', 'Nationality', 'Gender',
       'Graduation Year', 'EducationLevel', 'GeneralMajorName',
       'NarrowMajorName', 'MajorNameByClassification',
       'ISCOOccupationDescription', 'PeriodToEmployment'],
      dtype='object')

In [27]:
df['EducationLevel'].unique()

array(['بكالوريوس', 'دبلوم عالي', 'دبلوم متوسط', 'دبلوم مشارك', 'دكتوراه',
       'زمالة', 'غير معرف', 'ماجستير'], dtype=object)

In [28]:
df.to_json(r"C:\Users\sanav\work\D2R\NLO\University\public\output\translated_full_data_arabic.json", orient="records", indent=4, force_ascii=False)
df.to_excel(r"C:\Users\sanav\work\D2R\NLO\University\public\output\translated_full_data_arabic.xlsx", index=False)

In [32]:
df['Gender'].unique()

array(['أنثى', 'ذكر', 'غير معرف'], dtype=object)

In [30]:
df.shape

(253883, 11)

In [31]:
data.shape

(253883, 16)

EXTRA CODES

In [11]:
import pandas as pd
import json

# Read the Excel file with all sheets
excel_data = pd.read_excel(r"C:\Users\sanav\work\D2R\NLO\University\public\data\reflected.xlsx", sheet_name=None)

# Create a dictionary to store all translations
translation_dict = {}

# Process each sheet
for sheet_name, sheet_data in excel_data.items():
    # Convert the two columns into a dictionary
    translation_dict[sheet_name] = {
        str(key).strip(): str(value).strip() if isinstance(value, str) else value
        for key, value in zip(sheet_data.iloc[:, 0], sheet_data.iloc[:, 1])
    }

# Save as JSON file
with open(r"C:\Users\sanav\work\D2R\NLO\University\public\data\translations.json", 'w', encoding='utf-8') as f:
    json.dump(translation_dict, f, ensure_ascii=False, indent=4)

In [3]:
df = pd.read_excel(r"C:\Users\sanav\work\D2R\NLO\University\public\output\translated_full_data.xlsx", dtype='object')

In [22]:
data = pd.read_excel(
    r"C:\Users\sanav\work\D2R\NLO\University\public\data\UNI-IND-2022-Jadarat.xlsx",
    dtype="object",
)
data.dropna(inplace=True)

data = data[
    [
        "IndicatorDescription",
        "IndicatorValue",
        "Nationality",
        "Gender",
        "Graduation Year",
        "EducationLevel",
        "GeneralMajorName",
        "NarrowMajorName",
        "MajorNameByClassification",
        # "GOSIoccupationDescription",
        "ISCOOccupationDescription",
        "PeriodToEmployment",
    ]
]

In [34]:
df1 = data.copy()
df1 = df1[df1['Nationality'] == 'سعودي']
df1.reset_index(inplace=True, drop=True)

In [ ]:
df1.shape

In [ ]:
df1.columns

In [ ]:
df1['IndicatorDescription'].unique()

In [ ]:
df1['GeneralMajorName'].nunique()

In [ ]:
df1['GeneralMajorName'].unique()

In [43]:
df1['GeneralMajorName'] = df1['GeneralMajorName'].str.strip()

In [ ]:
df1['GeneralMajorName'].nunique()

In [27]:
# Code for changing values from Arabic to English in original data
translated_results = {}
excel_data = pd.read_excel(r"C:\Users\sanav\work\D2R\NLO\University\public\data\reflected.xlsx", sheet_name=None)

for sheet_name, sheet_data in excel_data.items():
    translated_results[sheet_name] = {
        str(key).strip(): str(value).strip() if isinstance(value, str) else value
        for key, value in zip(sheet_data.iloc[:, 0], sheet_data.iloc[:, 1])
    }

for column, translation_dict in translated_results.items():
    if column in df.columns:
        if df1[column].dtype == 'object':
            df1[column] = df1[column].str.strip()
        df1[column] = df1[column].map(translation_dict).fillna(df[column])

In [ ]:
df1.columns

In [ ]:
df.columns

In [ ]:
df1['NarrowMajorName'].nunique()

In [ ]:
df['NarrowMajorName'].nunique()

In [70]:
l1 = df['MajorNameByClassification'].unique().tolist()

In [ ]:
len(l1)

In [47]:
df2 = pd.read_excel(r"C:\Users\sanav\work\D2R\NLO\University\public\output\translated_full_data_arabic.xlsx", dtype='object')

In [73]:
l2 = df2['MajorNameByClassification'].unique().tolist()

In [ ]:
len(l2)

In [75]:
IGNORE_VALUES = [
    'Unclassified',
    'Unknown programs',
    'Unclassified programs',
    'Unknown Specializations',
    '0',
    "غير محدد",
    "غير معرف",
    "برامج غير محددة",
    "برامج غير معروفة",
    "برامج غير معروفة",
    "تخصصات غير معروفة"
]

In [ ]:
for i in l2:
    if i in IGNORE_VALUES:
        print(i)

In [ ]:
trs = GoogleTranslator(source='auto', target='ar')
print(trs.translate("Bachelor"))
# print(trs.translate("الأعمال والإدارة والقانون"))

In [ ]:
if "بكالوريوس" == 'بكالوريوس':
    print("Yes")

In [80]:
general_major = 'برامج غير محددة في الأعمال والإدارة والقانون'

In [ ]:
if any(ignore in str(general_major) for ignore in IGNORE_VALUES):
    print("Yes")

In [121]:
testing_data = pd.read_excel(r"C:\Users\sanav\work\D2R\NLO\University\public\data\data for the 13 majors.xlsx", dtype='str')

In [ ]:
testing_data

In [123]:
testing_data.rename(columns={'Unnamed: 0': 'GeneralMajorName'}, inplace=True)

In [ ]:
testing_data

In [125]:
translated_results = {}
excel_data = pd.read_excel(r"C:\Users\sanav\work\D2R\NLO\University\public\data\reflected.xlsx", sheet_name=None)

for sheet_name, sheet_data in excel_data.items():
    translated_results[sheet_name] = {
        str(key).strip(): str(value).strip() if isinstance(value, str) else value
        for key, value in zip(sheet_data.iloc[:, 0], sheet_data.iloc[:, 1])
    }

In [ ]:
translated_results['GeneralMajorName']

In [127]:
for column, translation_dict in translated_results.items():
    if column in testing_data.columns:
        if testing_data[column].dtype == 'object':
            testing_data[column] = testing_data[column].str.strip()
        testing_data[column] = testing_data[column].map(translation_dict).fillna(testing_data[column])

In [ ]:
testing_data

In [129]:
testing_data.to_excel(r"C:\Users\sanav\work\D2R\NLO\University\public\output\testing_data.xlsx", sheet_name= 'GeneralMajorName', index=False)

In [4]:
import pandas as pd
import json

# Read the job seekers data
job_seekers_df = pd.read_excel(r"C:\Users\sanav\work\D2R\NLO\University\public\data\UNI-IND-2022-Jadarat.xlsx", dtype='str')
job_seekers_df['IndicatorValue'] = pd.to_numeric(job_seekers_df['IndicatorValue'], errors='coerce')

# Translate the data
excel_data = pd.read_excel(r"C:\Users\sanav\work\D2R\NLO\University\public\data\reflected.xlsx", sheet_name=None)
translated_results = {}

for sheet_name, sheet_data in excel_data.items():
    translated_results[sheet_name] = {
        str(key).strip(): str(value).strip() if isinstance(value, str) else value
        for key, value in zip(sheet_data.iloc[:, 0], sheet_data.iloc[:, 1])
    }

for column, translation_dict in translated_results.items():
    if column in job_seekers_df.columns:
        if job_seekers_df[column].dtype == 'object':
            job_seekers_df[column] = job_seekers_df[column].str.strip()
        job_seekers_df[column] = job_seekers_df[column].map(translation_dict).fillna(job_seekers_df[column])

# Read the main data for other calculations
with open(r"C:\Users\sanav\work\D2R\NLO\University\public\output\translated_full_data.json", 'r', encoding='utf-8') as f:
    main_data = json.load(f)
df = pd.DataFrame(main_data)
df['IndicatorValue'] = pd.to_numeric(df['IndicatorValue'], errors='coerce')

# Create a DataFrame for NarrowMajor analysis
narrow_major_data = []

for narrow_major in df['NarrowMajorName'].unique():
    if pd.isna(narrow_major):
        continue
        
    narrow_major_df = df[df['NarrowMajorName'] == narrow_major]
    job_seekers_narrow_df = job_seekers_df[job_seekers_df['NarrowMajorName'] == narrow_major]
    
    # Calculate total graduates
    total_graduates = narrow_major_df[narrow_major_df['IndicatorDescription'] == 'Number of Graduates']['IndicatorValue'].sum()
    
    # Calculate employment rate
    total_employed = narrow_major_df[
        (narrow_major_df['IndicatorDescription'] == 'Number of graduates who are employed before graduation') |
        (narrow_major_df['IndicatorDescription'] == 'Number of graduates who are employed after graduation')
    ]['IndicatorValue'].sum()
    employment_rate = round((total_employed / total_graduates * 100), 2) if total_graduates > 0 else 0
    
    # Calculate average salary
    total_salary = narrow_major_df[narrow_major_df['IndicatorDescription'] == 'Total salaries of employees after graduation']['IndicatorValue'].sum()
    employed_with_salary = narrow_major_df[narrow_major_df['IndicatorDescription'] == 'Number of graduates with salary who are employed after graduation']['IndicatorValue'].sum()
    average_salary = round(total_salary / employed_with_salary) if employed_with_salary > 0 else 0
    
    # Calculate time to employment
    total_days = narrow_major_df[narrow_major_df['IndicatorDescription'] == 'Total number of days until the first job']['IndicatorValue'].sum()
    employed_after_grad = narrow_major_df[narrow_major_df['IndicatorDescription'] == 'Number of graduates who are employed after graduation']['IndicatorValue'].sum()
    time_to_employ = round((total_days / employed_after_grad) / 30, 1) if employed_after_grad > 0 else 0
    
    # Calculate job seekers from job_seekers_df
    job_seekers = job_seekers_narrow_df[job_seekers_narrow_df['IndicatorDescription'] == 'Number of Job Seekers']['IndicatorValue'].sum()
    
    narrow_major_data.append({
        'NarrowMajorName': narrow_major,
        'Grad No': int(total_graduates),
        'Employ Rate': float(employment_rate),
        'Salary Average': average_salary,
        'time to employ': time_to_employ,
        'job seekers': int(job_seekers)
    })

# Convert to DataFrame
narrow_major_df = pd.DataFrame(narrow_major_data)

# Add Grand Total row
# Add Grand Total row
total_grads = narrow_major_df['Grad No'].sum()
total_employed = narrow_major_df.apply(lambda x: x['Grad No'] * x['Employ Rate'], axis=1).sum()
total_salary = narrow_major_df.apply(lambda x: x['Salary Average'] * (x['Grad No'] * x['Employ Rate']), axis=1).sum()
total_time = narrow_major_df.apply(lambda x: x['time to employ'] * (x['Grad No'] * x['Employ Rate']), axis=1).sum()

grand_total = {
    'NarrowMajorName': 'Grand Total',
    'Grad No': total_grads,
    'Employ Rate': round(total_employed / total_grads * 100, 2),
    'Salary Average': round(total_salary / total_employed),
    'time to employ': round(total_time / total_employed, 1),
    'job seekers': narrow_major_df['job seekers'].sum()
}

narrow_major_df = pd.concat([narrow_major_df, pd.DataFrame([grand_total])], ignore_index=True)

# Save to Excel
# Save to Excel
try:
    with pd.ExcelWriter(r"C:\Users\sanav\work\D2R\NLO\University\public\output\testing_data.xlsx", 
                        mode='a', 
                        engine='openpyxl',
                        if_sheet_exists='replace') as writer:
        narrow_major_df.to_excel(writer, sheet_name='NarrowMajorName', index=False)
except ValueError:
    # If mode='a' fails, try writing a new file
    with pd.ExcelWriter(r"C:\Users\sanav\work\D2R\NLO\University\public\output\testing_data.xlsx", 
                        engine='openpyxl') as writer:
        narrow_major_df.to_excel(writer, sheet_name='NarrowMajorName', index=False)

In [ ]:
narrow_major_df

In [ ]:
import pandas as pd
from openpyxl.styles import Font

formulas = [
    {
        'Level': 'Overall',
        'Metric': 'Total Graduates',
        'Formula': '"Number of Graduates"',
        'Description': 'Sum of all graduates'
    },
    {
        'Level': 'Overall',
        'Metric': 'Employment Rate',
        'Formula': '("Number of graduates who are employed before graduation" + "Number of graduates who are employed after graduation") / "Number of Graduates" × 100',
        'Description': 'Overall employment rate'
    },
    {
        'Level': 'Overall',
        'Metric': 'Time to Employment (Months)',
        'Formula': '("Total number of days until the first job" / "Number of graduates who are employed after graduation") / 30',
        'Description': 'Average months to find employment'
    },
    {
        'Level': 'Overall',
        'Metric': 'Average Salary',
        'Formula': '"Total salaries of employees after graduation" / "Number of graduates with salary who are employed after graduation"',
        'Description': 'Overall average salary'
    },
    {
        'Level': 'Overall',
        'Metric': 'Job Seekers',
        'Formula': '"Number of Job Seekers"',
        'Description': 'Total number of job seekers'
    },
    {
        'Level': 'General Major',
        'Metric': 'Total Graduates',
        'Formula': '"Number of Graduates" WHERE GeneralMajorName = X',
        'Description': 'Sum of graduates in specific general major'
    },
    {
        'Level': 'General Major',
        'Metric': 'Employment Rate',
        'Formula': '("Number of graduates who are employed before graduation" + "Number of graduates who are employed after graduation") / "Number of Graduates" × 100 WHERE GeneralMajorName = X',
        'Description': 'Employment rate for specific general major'
    },
    {
        'Level': 'General Major',
        'Metric': 'Time to Employment (Months)',
        'Formula': '("Total number of days until the first job" / "Number of graduates who are employed after graduation") / 30 WHERE GeneralMajorName = X',
        'Description': 'Average months to find employment for specific general major'
    },
    {
        'Level': 'General Major',
        'Metric': 'Average Salary',
        'Formula': '"Total salaries of employees after graduation" / "Number of graduates with salary who are employed after graduation" WHERE GeneralMajorName = X',
        'Description': 'Average salary for specific general major'
    },
    {
        'Level': 'General Major',
        'Metric': 'Job Seekers',
        'Formula': '"Number of Job Seekers" WHERE GeneralMajorName = X',
        'Description': 'Number of job seekers in specific general major'
    },
    {
        'Level': 'Narrow Major',
        'Metric': 'Total Graduates',
        'Formula': '"Number of Graduates" WHERE NarrowMajorName = Y',
        'Description': 'Sum of graduates in specific narrow major'
    },
    {
        'Level': 'Narrow Major',
        'Metric': 'Employment Rate',
        'Formula': '("Number of graduates who are employed before graduation" + "Number of graduates who are employed after graduation") / "Number of Graduates" × 100 WHERE NarrowMajorName = Y',
        'Description': 'Employment rate for specific narrow major'
    },
    {
        'Level': 'Narrow Major',
        'Metric': 'Time to Employment (Months)',
        'Formula': '("Total number of days until the first job" / "Number of graduates who are employed after graduation") / 30 WHERE NarrowMajorName = Y',
        'Description': 'Average months to find employment for specific narrow major'
    },
    {
        'Level': 'Narrow Major',
        'Metric': 'Average Salary',
        'Formula': '"Total salaries of employees after graduation" / "Number of graduates with salary who are employed after graduation" WHERE NarrowMajorName = Y',
        'Description': 'Average salary for specific narrow major'
    },
    {
        'Level': 'Narrow Major',
        'Metric': 'Job Seekers',
        'Formula': '"Number of Job Seekers" WHERE NarrowMajorName = Y',
        'Description': 'Number of job seekers in specific narrow major'
    }
]

# Convert to DataFrame
formulas_df = pd.DataFrame(formulas)

# Save to Excel with formatted columns
with pd.ExcelWriter(r"C:\Users\sanav\work\D2R\NLO\University\public\output\metrics_formulas.xlsx", engine='openpyxl') as writer:
    formulas_df.to_excel(writer, sheet_name='Formulas', index=False)
    
    # Get the workbook and the worksheet
    workbook = writer.book
    worksheet = writer.sheets['Formulas']
    
    # Adjust column widths
    for idx, col in enumerate(formulas_df.columns):
        max_length = max(
            formulas_df[col].astype(str).apply(len).max(),
            len(col)
        )
        worksheet.column_dimensions[chr(65 + idx)].width = min(max_length + 2, 100)
    
    # Add bold formatting to header row
    bold_font = Font(bold=True)
    for cell in worksheet[1]:
        cell.font = bold_font

print("Metrics formulas have been saved to metrics_formulas.xlsx")

In [ ]:
df.shape

In [ ]:
df.columns